In [1]:
import numpy as np
import pandas as pd

In [13]:
av_sales_df = pd.read_csv('/Users/adityan/Work/ds_practice/av_sales/Avantor_sales_data.csv')

In [14]:
av_sales_df.head()

,Country,Customer Group,Customer Name,Product Group,Product Sub Group,Sales\n(USD),Sold Quantity\n(USD),Cost\n(USD),MRP\n(USD)
0,UAE,113-BioMérieux,BIOMERIEUX SA,10-Chemicals,1004-Inorganic Compounds & Salts,"6,56,181","4,317","5,38,631","21,28,753"
1,Korea,143-GE Healthcare,GE Healthcare AS,10-Chemicals,1007-Organic Chemistry,"5,14,744",328,"4,20,478","5,18,358"
2,UAE,536-Bio-Rad,BIO RAD,10-Chemicals,1011-Customized Products,"3,61,807",275,"1,59,117","3,61,807"
3,Korea,104-Zoetis,PHARMAQ Analytiq AS,10-Chemicals,1020-Genomics & Molecular Diagnostics,"3,23,768",151,"2,15,055","5,34,229"
4,China,108-AstraZeneca,ASTRAZENECA AB,10-Chemicals,1020-Genomics & Molecular Diagnostics,"3,21,576",616,"2,91,265","3,22,934"


In [25]:
av_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58104 entries, 0 to 58103
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Country                        58104 non-null  object
 1   Customer Group                 58104 non-null  object
 2   Customer Name                  58104 non-null  object
 3   Product Group                  58104 non-null  object
 4   Product Sub Group              58104 non-null  object
 5   Sales                          58104 non-null  object
 6   Sold Quantity                  58104 non-null  object
 7   Cost                           58104 non-null  object
 8   MRP                            58104 non-null  object
 9   Product Sub Group ID           58104 non-null  object
 10  Product Sub Group Description  58104 non-null  object
dtypes: object(11)
memory usage: 4.9+ MB


In [15]:
# correct the columns name:
av_sales_df.rename(columns={'Sales\n(USD)':'Sales','Sold Quantity\n(USD)':'Sold Quantity','Cost\n(USD)':'Cost','MRP\n(USD)':'MRP'}
                   ,inplace=True)
av_sales_df.head()

,Country,Customer Group,Customer Name,Product Group,Product Sub Group,Sales,Sold Quantity,Cost,MRP
0,UAE,113-BioMérieux,BIOMERIEUX SA,10-Chemicals,1004-Inorganic Compounds & Salts,"6,56,181","4,317","5,38,631","21,28,753"
1,Korea,143-GE Healthcare,GE Healthcare AS,10-Chemicals,1007-Organic Chemistry,"5,14,744",328,"4,20,478","5,18,358"
2,UAE,536-Bio-Rad,BIO RAD,10-Chemicals,1011-Customized Products,"3,61,807",275,"1,59,117","3,61,807"
3,Korea,104-Zoetis,PHARMAQ Analytiq AS,10-Chemicals,1020-Genomics & Molecular Diagnostics,"3,23,768",151,"2,15,055","5,34,229"
4,China,108-AstraZeneca,ASTRAZENECA AB,10-Chemicals,1020-Genomics & Molecular Diagnostics,"3,21,576",616,"2,91,265","3,22,934"


In [62]:
print(av_sales_df.columns)

Index(['Country', 'Customer Group', 'Customer Name', 'Product Group',
       'Product Sub Group', 'Sales', 'Sold Quantity', 'Cost', 'MRP',
       'Product Sub Group ID', 'Product Sub Group Description'],
      dtype='object')


## 1. Split Product Sub group into Product Sub group ID and Description

In [43]:
av_sales_df['Product Sub Group ID'] = av_sales_df['Product Sub Group'].str.split('-').str[0]
av_sales_df['Product Sub Group Description'] = av_sales_df['Product Sub Group'].str.split('-').str[1]
av_sales_df.head()

,Country,Customer Group,Customer Name,Product Group,Product Sub Group,Sales,Sold Quantity,Cost,MRP,Product Sub Group ID,Product Sub Group Description
0,UAE,113-BioMérieux,BIOMERIEUX SA,10-Chemicals,1004-Inorganic Compounds & Salts,"6,56,181","4,317","5,38,631","21,28,753",1004,Inorganic Compounds & Salts
1,Korea,143-GE Healthcare,GE Healthcare AS,10-Chemicals,1007-Organic Chemistry,"5,14,744",328,"4,20,478","5,18,358",1007,Organic Chemistry
2,UAE,536-Bio-Rad,BIO RAD,10-Chemicals,1011-Customized Products,"3,61,807",275,"1,59,117","3,61,807",1011,Customized Products
3,Korea,104-Zoetis,PHARMAQ Analytiq AS,10-Chemicals,1020-Genomics & Molecular Diagnostics,"3,23,768",151,"2,15,055","5,34,229",1020,Genomics & Molecular Diagnostics
4,China,108-AstraZeneca,ASTRAZENECA AB,10-Chemicals,1020-Genomics & Molecular Diagnostics,"3,21,576",616,"2,91,265","3,22,934",1020,Genomics & Molecular Diagnostics


## 2. Top 5 and Bottom 5 Sold Quantity Sum at Product Sub Group Description level with Total Sales values

In [55]:
product_sub_grp_df = av_sales_df[['Product Sub Group ID','Product Sub Group Description','Sold Quantity','Sales']]
product_sub_grp_df.loc[:,'Sold Quantity'] = product_sub_grp_df.loc[:,'Sold Quantity'].str.replace(',','')
product_sub_grp_df.loc[:,'Sales'] = product_sub_grp_df.loc[:,'Sales'].str.replace(',','')
dtype_dict = {'Product Sub Group ID': str
              ,'Product Sub Group Description': str
              ,'Sold Quantity': float
              ,'Sales': float
             }
product_sub_grp_df = product_sub_grp_df.astype(dtype_dict)

product_sub_grp_df = product_sub_grp_df.groupby(['Product Sub Group ID','Product Sub Group Description']).agg(
                                        Total_Sold_Quantity=('Sold Quantity', np.sum)
                                        ,Total_Sales=('Sales', np.sum)
).reset_index()
#product_sub_grp_df

product_sub_grp_df_final = pd.concat([product_sub_grp_df.sort_values(['Total_Sold_Quantity'],ascending=False).head(5),product_sub_grp_df.sort_values(['Total_Sold_Quantity'],ascending=True).head(5)]
                                     ,axis=0)
product_sub_grp_df_final

/var/folders/kh/ftflw6l10qs5cc3hvld2zmjh0000gq/T/ipykernel_56552/846297837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_sub_grp_df.loc[:,'Sold Quantity'] = product_sub_grp_df.loc[:,'Sold Quantity'].str.replace(',','')
/var/folders/kh/ftflw6l10qs5cc3hvld2zmjh0000gq/T/ipykernel_56552/846297837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_sub_grp_df.loc[:,'Sales'] = product_sub_grp_df.loc[:,'Sales'].str.replace(',','')


,Product Sub Group ID,Product Sub Group Description,Total_Sold_Quantity,Total_Sales
0,1001,Solvents,487641.0,20421961.0
14,1015,Microbiology,116548.0,6752097.0
4,1005,Specialty Reagents,114499.0,5194863.0
15,1016,Diagnostic Reagents,113487.0,2874956.0
24,1099,Various Chemicals,103569.0,5626437.0
23,1031,Fire Assay Chemicals,14.0,2422.0
8,1009,Various Chemicals,68.0,833.0
7,1008,Biochemicals,70.0,7826.0
17,1018,Components related to Chrom consumables,312.0,98614.0
13,1014,Testkits Food,772.0,93070.0


## 3. List the countries in the order of high to low distinct Product Sub Group Count

In [61]:
product_sub_group_count_w_countries = av_sales_df.groupby(['Country']).agg(
                                            {'Product Sub Group ID':pd.Series.nunique}
                                            ).reset_index()
product_sub_group_count_w_countries = product_sub_group_count_w_countries.sort_values(['Product Sub Group ID'],ascending=False)
product_sub_group_count_w_countries

,Country,Product Sub Group ID
0,Australia,25
1,China,23
5,UAE,23
2,India,22
3,Japan,22
4,Korea,21


## 4. List only Top 2 Product Sub Group from the Product Group with sum of Sales, Sold Quantity, Cost and MRP

In [65]:
top2_sub_group = av_sales_df[['Product Group','Product Sub Group ID', 'Product Sub Group Description'
                              ,'Sales', 'Sold Quantity', 'Cost', 'MRP']]
top2_sub_group.loc[:,'Sold Quantity'] = top2_sub_group.loc[:,'Sold Quantity'].str.replace(',','')
top2_sub_group.loc[:,'Sales'] = top2_sub_group.loc[:,'Sales'].str.replace(',','')
top2_sub_group.loc[:,'Cost'] = top2_sub_group.loc[:,'Sales'].str.replace(',','')
top2_sub_group.loc[:,'MRP'] = top2_sub_group.loc[:,'Sales'].str.replace(',','')

dtype_dict_2 = {'Cost': float
              ,'MRP': float
              ,'Sold Quantity': float
              ,'Sales': float
             }
top2_sub_group = top2_sub_group.astype(dtype_dict_2)

top2_sub_group

/var/folders/kh/ftflw6l10qs5cc3hvld2zmjh0000gq/T/ipykernel_56552/1329935605.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2_sub_group.loc[:,'Sold Quantity'] = top2_sub_group.loc[:,'Sold Quantity'].str.replace(',','')
/var/folders/kh/ftflw6l10qs5cc3hvld2zmjh0000gq/T/ipykernel_56552/1329935605.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2_sub_group.loc[:,'Sales'] = top2_sub_group.loc[:,'Sales'].str.replace(',','')
/var/folders/kh/ftflw6l10qs5cc3hvld2zmjh0000gq/T/ipykernel_56552/1329935605.

,Product Group,Product Sub Group ID,Product Sub Group Description,Sales,Sold Quantity,Cost,MRP
0,10-Chemicals,1004,Inorganic Compounds & Salts,656181.0,4317.0,656181.0,656181.0
1,10-Chemicals,1007,Organic Chemistry,514744.0,328.0,514744.0,514744.0
2,10-Chemicals,1011,Customized Products,361807.0,275.0,361807.0,361807.0
3,10-Chemicals,1020,Genomics & Molecular Diagnostics,323768.0,151.0,323768.0,323768.0
4,10-Chemicals,1020,Genomics & Molecular Diagnostics,321576.0,616.0,321576.0,321576.0
...,...,...,...,...,...,...,...
58099,10-Chemicals,1012,Testkits Diagnostics,-9536.0,1.0,-9536.0,-9536.0
58100,10-Chemicals,1001,Solvents,-11468.0,-1321.0,-11468.0,-11468.0
58101,10-Chemicals,1011,Customized Products,-13644.0,-24.0,-13644.0,-13644.0
58102,10-Chemicals,1012,Testkits Diagnostics,-18540.0,-1.0,-18540.0,-18540.0


In [73]:
top2_sub_group_final = top2_sub_group.groupby(['Product Group','Product Sub Group Description']).agg(
                                            {'Product Sub Group ID':pd.Series.nunique
                                            ,'Sales': np.sum
                                            ,'Sold Quantity': np.sum
                                            ,'Cost': np.sum
                                            ,'MRP': np.sum
                                            }).reset_index().sort_values(['Product Sub Group ID'],ascending=False).head(2)
top2_sub_group_final

,Product Group,Product Sub Group Description,Product Sub Group ID,Sales,Sold Quantity,Cost,MRP
22,10-Chemicals,Various Chemicals,2,5627270.0,103637.0,5627270.0,5627270.0
1,10-Chemicals,Biochemicals,2,2939022.0,22154.0,2939022.0,2939022.0
